In [27]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
image_path = "./datasets/mbappe/gettyimages-1477251906-612x612.jpg"
img = cv2.imread(image_path)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(img.shape)
print(gray.shape)
plt.imshow(img)

In [ ]:
plt.imshow(gray)

In [ ]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# Extract the face
faces = face_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
)
faces

In [ ]:
# extract the eyes
for x, y, w, h in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    roi_gray = gray[y : y + h, x : x + w]
    roi_color = img[y : y + h, x : x + w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for ex, ey, ew, eh in eyes:
        cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

In [ ]:
# Convert BGR image to RGB for matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the output with matplotlib
plt.figure()
plt.imshow(img_rgb, cmap="gray")
# plt.imshow(img_rgb)
# plt.axis("off")
plt.show()

In [ ]:
plt.imshow(roi_color, cmap="gray")

In [ ]:
import cv2
import matplotlib.pyplot as plt

folder_name = "mbappe"
image_name = crop_images_dict[folder_name][0]
image_path = os.path.join("./face_crop", folder_name, image_name)

# Read the image
img = cv2.imread(image_path)

# Convert BGR to RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image
plt.imshow(img_rgb)
plt.axis("off")
plt.show()

## Data Preprocessing

In [ ]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:



# create a dictionary with the folder name as the key and the images as as list of value
def create_crop_images_dict(base_folder):
    crop_images_dict = {}
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            crop_images_dict[folder_name] = [
                img for img in os.listdir(folder_path) if img.endswith(".jpg")
            ]
    return crop_images_dict

In [ ]:
crop_images_dict = create_crop_images_dict("./face_crop/")
crop_images_dict.keys(), crop_images_dict["oshoala"][0]

In [ ]:
len(crop_images_dict["oshoala"]), len(crop_images_dict["messi"])

In [ ]:
# Split the key, values into X and labels
base_path = "./face_crop/"

X = []  # Image data
y = []  # Labels

for label, image_files in crop_images_dict.items():
    for image_file in image_files:
        image_path = os.path.join(base_path, label, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, (100, 100))  # Resize to uniform size
            X.append(img)
            y.append(label)

# convert to numpy array
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
# Encode y label
# Mapping dictionary
label_mapping = {"oshoala": 0, "mbappe": 1, "ronaldo": 2, "messi": 3, "morgan": 4}
y_encode = [label_mapping[label] for label in y]
label = np.array(y_encode)

In [ ]:
# Normalize and Augment Data

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, label, test_size=0.2, random_state=42
)

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)  # Only rescaling for validation data

# Create generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = val_datagen.flow(X_val, y_val, batch_size=32)

## Build our CNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Build model
model = Sequential()

# Convolutional layer
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(100, 100, 3)))
model.add(MaxPooling2D(2, 2))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))

# Flatten the results to feed into a dense layer
model.add(Flatten())

# 128 neuron hidden layer
model.add(Dense(128, activation="relu"))

# Dropout layer to reduce overfitting
model.add(Dropout(0.5))

# Output layer
# Use softmax for multi-class classification, sigmoid for binary
model.add(Dense(5, activation="softmax"))

In [ ]:
# Compile the model

optimizer = Adam(learning_rate=0.001)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer, 
    metrics=["accuracy"]
)

# Model summary
model.summary()

In [ ]:
# Train the Model
# Set the number of epochs and batch size
epochs = 50 # This maybe be finetune

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,  # Number of steps per epoch
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(X_val) // 32,  # Number of validation steps
)

In [ ]:
# Evaluate and Save the Model
model.evaluate(validation_generator)

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint = ModelCheckpoint(
    "best_model.h5",  # Filename to save model
    monitor="val_loss",  # Monitor validation loss
    verbose=1,
    save_best_only=True,  # Only save when the monitored metric has improved
    mode="min",  # Mode 'min' for loss, 'max' for accuracy
)

# Train the model with the callback
epochs = 50  # This maybe be finetune
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,  # Number of steps per epoch
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(X_val) // 32,  # Number of validation steps
    callbacks=[checkpoint],
)

In [ ]:
# Load and Evaluate best Model
model = tf.keras.models.load_model("best_model.h5")
model.evaluate(validation_generator)

### Test our Model By Predicting New Image

In [29]:
# Load Model
import tensorflow as tf
model = model = tf.keras.models.load_model("best_model.h5")

In [30]:
# Prepare th image
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (100, 100))
    img = img / 255.0
    return img.reshape(1, 100, 100, 3)  # Reshape for the model


**label_mapping**
* oshoala: 0
* mbappe: 1
* ronaldo: 2
* messi: 3
* morgan: 4


In [31]:
# Make Predictions
new_image = preprocess_image(
    "./face_crop/messi/face_gettyimages-82525445-612x612_2.jpg"
)
prediction = model.predict(new_image)
predicted_class = np.argmax(prediction, axis=1)  # get the max value in the predictions
predicted_class, prediction

1/1 [==============================] - 0s 64ms/step


(array([4]),
 array([[0.00208999, 0.00261326, 0.01020569, 0.09974755, 0.88534343]],
       dtype=float32))